# Scrape BC Liquor Store Website

### Import libraries

In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm
import csv

### Functions to scrape information from the website

In [13]:
def find_tags(var,begin,end):
    price =[]
    name=[]
    page =[]
    country =[]
    sku_link = []
    SCROLL_PAUSE_TIME = 5
    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    
    for pagenum in tqdm(range(begin,end)):
        driver.get('https://www.bcliquorstores.com/product-catalogue?category=%s&sort=name.raw:asc&page=%d' % (var,pagenum))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        
        name_i = [i.get_text() for i in soup.find_all('h3', class_ = "product-name")]
        country_i = [i.get_text() for i in soup.find_all('span', class_ = "country-code")]
        sku_link_i = [i.get('href') for i in soup.find_all('a')][118:214:4]
        price_i= [i.get_text() for i in soup.find_all('div', class_ = "product-price")]
        
        #append to the final list
        name.append(name_i)
        country.append(country_i)
        sku_link.append(sku_link_i)
        price.append(price_i)
    
    return name, country, sku_link, price

In [14]:
name, country, sku_link, price = find_tags('beer', 1, 22)
name = [i for m in name for i in m]
country = [i for m in country for i in m]
sku_link = [i for m in sku_link for i in m]
price= [i for m in price for i in m]

100%|██████████| 21/21 [01:58<00:00,  5.66s/it]


In [17]:
print(len(name))
print(len(country))
print(len(sku_link))
print(len(price))

502
502
504
502


In [19]:
sku_link

['/product/772335',
 '/product/880930',
 '/product/837591',
 '/product/90837',
 '/product/23859',
 '/product/588681',
 '/product/687251',
 '/product/638759',
 '/product/439950',
 '/product/541201',
 '/product/95003',
 '/product/20555',
 '/product/994145',
 '/product/198948',
 '/product/176909',
 '/product/193590',
 '/product/849752',
 '/product/79746',
 '/product/204407',
 '/product/127282',
 '/product/676437',
 '/product/215577',
 '/product/9209',
 '/product/322362',
 '/product/101624',
 '/product/183512',
 '/product/162057',
 '/product/82382',
 '/product/394536',
 '/product/483941',
 '/product/675462',
 '/product/233406',
 '/product/12757',
 '/product/132902',
 '/product/495283',
 '/product/488379',
 '/product/222794',
 '/product/848713',
 '/product/82239',
 '/product/151710',
 '/product/102114',
 '/product/34033',
 '/product/772756',
 '/product/328419',
 '/product/479857',
 '/product/91626',
 '/product/276188',
 '/product/102035',
 '/product/234334',
 '/product/102622',
 '/product/1

In [20]:
df = pd.DataFrame({"name": name, "country":country, "price": price, "sku_link": sku_link[:-2]})

In [21]:
df.head()

,name,country,price,sku_link
0,33 ACRES OF LIFE LAGER,Canada,$12.29,/product/772335
1,33 ACRES OF NIRVANA 6 PACK,Canada,$13.49,/product/880930
2,33 ACRES OF OCEAN PALE ALE,Canada,$12.29,/product/837591
3,33 ACRES OF SUNSHINE BLANCHE,Canada,$12.29,/product/90837
4,33 ACRES OF SUNSHINE CAN,Canada,$21.99,/product/23859


In [24]:
def product_details(links):
   
    alc = []
    style = []
    beer_type =[]
    desc = []
    price =[]
    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    
    for sku in tqdm(links):
        driver.get('https://www.bcliquorstores.com%s' % (sku))
        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        alc_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.alcoholPercentage + '%'" })]
        style_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.style" })]
        beer_type_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: $root.FormatProductType($data._source)" })]
        desc_i = [i.get_text() for i in soup.find_all('span', attrs={"data-bind": "text: _source.tastingDescription" })]
        price_i= [i.get_text() for i in soup.find_all('div', class_ = "product-price")]
        
        #append to the final list
        alc.append(alc_i)
        style.append(style_i)
        beer_type.append(beer_type_i)
        desc.append(desc_i)
        price.append(price_i)
    
    return alc,style,beer_type,desc, price

In [25]:
alc,style,beer_type,desc, price = product_details(df.sku_link)

100%|██████████| 502/502 [48:11<00:00,  5.76s/it]


In [26]:
alc

[['4.8%', '4.8%'],
 ['7%', '7%'],
 ['5.6%', '5.6%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['6.4%', '6.4%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5.6%', '5.6%'],
 ['5.6%', '5.6%'],
 ['5.6%', '5.6%'],
 ['5.6%', '5.6%'],
 ['5.4%', '5.4%'],
 ['4.7%', '4.7%'],
 ['7%', '7%'],
 ['7%', '7%'],
 ['5%', '5%'],
 ['4.7%', '4.7%'],
 ['5%', '5%'],
 ['6.7%', '6.7%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5.4%', '5.4%'],
 ['5.45%', '5.45%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['4.8%', '4.8%'],
 ['5.4%', '5.4%'],
 ['4.5%', '4.5%'],
 ['5%', '5%'],
 ['5.5%', '5.5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5.5%', '5.5%'],
 ['5.5%', '5.5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['4.8%', '4.8%'],
 ['5.5%', '5.5%'],
 ['5.2%', '5.2%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['7%', '7%'],
 ['6%', '6%'],
 ['5%', '5%'],
 ['5.2%', '5.2%'],
 ['4%', '4%'],
 ['0.5%', '0.5%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['4%', '4%'],
 ['3.8%', '3.8%'],
 ['5%', '5%'],
 ['5%', '5%'],
 ['5%', '5%']

In [30]:
# alcohol by volume
df['alc'] = [i for m in alc for i in m][0::2]

In [32]:

# beer style
style_final = []

for i in style:
    if i == []:
        style_final.append(['No info','No info'])
    else: 
        style_final.append(i)
df['style'] = [i for m in style_final for i in m][0::2]

In [37]:
# beer type
df['beer_type'] = [i for m in beer_type for i in m][0::2]
# price
df['price'] = [i for m in price for i in m][0::2]

In [40]:

#description
desc_final = []

for i in desc:
    if i == []:
        desc_final.append(['No info','No info'])
    else: 
        desc_final.append(i)
        
df['desc'] = [i for m in desc_final for i in m][0::2]

In [41]:
df.head()

,name,country,price,sku_link,alc,style,beer_type,desc
0,33 ACRES OF LIFE LAGER,Canada,$12.29,/product/772335,4.8%,California Common Or Steam,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulnes..."
1,33 ACRES OF NIRVANA 6 PACK,Canada,$13.49,/product/880930,7%,India Pale Ale,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved...
2,33 ACRES OF OCEAN PALE ALE,Canada,$12.29,/product/837591,5.6%,Pale Ale,Domestic Beer - BC Craft Beer,A full flavoured beer integrated with a distin...
3,33 ACRES OF SUNSHINE BLANCHE,Canada,$12.29,/product/90837,5%,Wheat,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this ro..."
4,33 ACRES OF SUNSHINE CAN,Canada,$21.99,/product/23859,5%,Wheat,Domestic Beer - BC Craft Beer,No info


In [76]:
# df.to_csv('bcstores.csv', index=False)